# mscoco mapの仕様
* idは多分見ていないが、evalを通した後にidの割り振りが変わる
* image_idは見ており、昇順にsortされる
* 画像順(image_id)順にソートはするが、画像内部ではsortしない

In [1]:
from copy import deepcopy

In [2]:
def create_dummy_coco_dataset(num_images=10, num_annotations=20,image_height=100, image_width=100, bbox_height=10, bbox_width=10):
    images = []
    annotations = []
    categories = [{"id": 1, "name": "object"}]

    for i in range(num_images):
        image = {
            "id": i,
            "width": image_width,
            "height": image_height,
            "file_name": f"image_{i}.jpg"
        }
        images.append(image)#

        assert image_height * image_width >= num_annotations * bbox_height * bbox_width, "Not enough space for annotations in the image."
        for j in range(num_annotations):
            height_index  = j // (image_width // bbox_width)
            width_index = j % (image_width // bbox_width)
            annotation = {
                "id": i * num_annotations + j,
                "image_id": i,
                "category_id": categories[0]["id"],
                "bbox": [width_index * bbox_width, height_index * bbox_height, bbox_width, bbox_height],
                "bbox_xyxy": [width_index * bbox_width, height_index * bbox_height, (width_index + 1) * bbox_width, (height_index + 1) * bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            }
            annotations.append(annotation)

    return {"images": images, "annotations": annotations, "categories": categories}

In [3]:
bbox_height = 10
bbox_width = 10
dummy_data = create_dummy_coco_dataset(num_images=1,num_annotations=100,bbox_height=bbox_height, bbox_width=bbox_width)
#dummy_data = create_dummy_coco_dataset(num_images=100,num_annotations=1,bbox_height=bbox_height, bbox_width=bbox_width)

In [4]:
dummy_images = dummy_data["images"]
dummy_annotations = dummy_data["annotations"]
dummy_categories = dummy_data["categories"]

In [5]:
annotations = dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0}


In [6]:
gt_dummy_annotations = deepcopy(dummy_annotations)
# for annotation in gt_dummy_annotations:
#     annotation["score"] = 1.0
annotations = gt_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0}


In [7]:

correct_dummy_annotations = deepcopy(dummy_annotations)
for annotation in correct_dummy_annotations:
    annotation["score"] = 1.0
annotations = correct_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])


{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0, 'score': 1.0}


In [8]:
#annotation["bbox"] = [annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][2], annotation["bbox"][3]]
tp50_fp50_dummy_annotations = deepcopy(dummy_annotations)
for index,annotation in enumerate(tp50_fp50_dummy_annotations):
    annotation["score"] = 1.0
    if index < 50:
        # annotation["score"] = 1.0
        pass
    else:
        # annotation["score"] = 0.0
        annotation["bbox"] = [annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][2]*0.1, annotation["bbox"][3]*0.1]
        annotation["bbox_xyxy"] = [annotation["bbox_xyxy"][0], annotation["bbox_xyxy"][1], annotation["bbox_xyxy"][0] + bbox_width*0.1, annotation["bbox_xyxy"][1] + bbox_height*0.1]
        annotation["area"] = annotation["bbox"][2] * annotation["bbox"][3]
annotations = tp50_fp50_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 1.0, 1.0], 'bbox_xyxy': [90, 90, 91.0, 91.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}


In [9]:
fp50_tp50_dummy_annotations = deepcopy(dummy_annotations)
for index,annotation in enumerate(fp50_tp50_dummy_annotations):
    annotation["score"] = 1.0
    #annotation["image_id"] = 99-index
    if index < 50:
        annotation["bbox"] = [annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][2]*0.1, annotation["bbox"][3]*0.1]
        annotation["bbox_xyxy"] = [annotation["bbox_xyxy"][0], annotation["bbox_xyxy"][1], annotation["bbox_xyxy"][0] + bbox_width*0.1, annotation["bbox_xyxy"][1] + bbox_height*0.1]
        annotation["area"] = annotation["bbox"][2] * annotation["bbox"][3]
annotations = fp50_tp50_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 1.0, 1.0], 'bbox_xyxy': [0, 0, 1.0, 1.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 1.0, 1.0], 'bbox_xyxy': [90, 0, 91.0, 1.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 1.0, 1.0], 'bbox_xyxy': [90, 10, 91.0, 11.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0, 'score': 1.0}


In [10]:
even_tp_od_fp_dummy_annotations = deepcopy(dummy_annotations)
for index,annotation in enumerate(even_tp_od_fp_dummy_annotations):
    annotation["score"] = 1.0
    if (index+1) % 2 == 0:
        pass
    else:
        annotation["bbox"] = [annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][2]*0.1, annotation["bbox"][3]*0.1]
        annotation["bbox_xyxy"] = [annotation["bbox_xyxy"][0], annotation["bbox_xyxy"][1], annotation["bbox_xyxy"][0] + bbox_width*0.1, annotation["bbox_xyxy"][1] + bbox_height*0.1]
        annotation["area"] = annotation["bbox"][2] * annotation["bbox"][3]
        
annotations = even_tp_od_fp_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 1.0, 1.0], 'bbox_xyxy': [0, 0, 1.0, 1.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0, 'score': 1.0}


In [11]:
fp_30_80_dummy_annotations = deepcopy(dummy_annotations)
for index,annotation in enumerate(fp_30_80_dummy_annotations):
    annotation["score"] = 1.0
    if (index+1) > 30 and (index+1) <= 80:
        annotation["bbox"] = [annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][2]*0.1, annotation["bbox"][3]*0.1]
        annotation["bbox_xyxy"] = [annotation["bbox_xyxy"][0], annotation["bbox_xyxy"][1], annotation["bbox_xyxy"][0] + bbox_width*0.1, annotation["bbox_xyxy"][1] + bbox_height*0.1]
        annotation["area"] = annotation["bbox"][2] * annotation["bbox"][3]
        
annotations = fp_30_80_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 9, 'image_id': 0, 'category_id': 1, 'bbox': [90, 0, 10, 10], 'bbox_xyxy': [90, 0, 100, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 19, 'image_id': 0, 'category_id': 1, 'bbox': [90, 10, 10, 10], 'bbox_xyxy': [90, 10, 100, 20], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 99, 'image_id': 0, 'category_id': 1, 'bbox': [90, 90, 10, 10], 'bbox_xyxy': [90, 90, 100, 100], 'area': 100, 'iscrowd': 0, 'score': 1.0}


In [12]:
random_tp50_fp50_dummy_annotations = deepcopy(tp50_fp50_dummy_annotations)
import random
random.seed(42)  # For reproducibility
random.shuffle(random_tp50_fp50_dummy_annotations)
# for index,annotation in enumerate(random_tp50_fp50_dummy_annotations):
#     annotation["id"] = index
annotations = random_tp50_fp50_dummy_annotations
print(annotations[0])
print(annotations[9])
print(annotations[19])
print(annotations[99])

{'id': 42, 'image_id': 0, 'category_id': 1, 'bbox': [20, 40, 10, 10], 'bbox_xyxy': [20, 40, 30, 50], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 78, 'image_id': 0, 'category_id': 1, 'bbox': [80, 70, 1.0, 1.0], 'bbox_xyxy': [80, 70, 81.0, 71.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}
{'id': 37, 'image_id': 0, 'category_id': 1, 'bbox': [70, 30, 10, 10], 'bbox_xyxy': [70, 30, 80, 40], 'area': 100, 'iscrowd': 0, 'score': 1.0}
{'id': 81, 'image_id': 0, 'category_id': 1, 'bbox': [10, 80, 1.0, 1.0], 'bbox_xyxy': [10, 80, 11.0, 81.0], 'area': 1.0, 'iscrowd': 0, 'score': 1.0}


In [13]:
sorted_random_tp50_fp50_dummy_annotations = sorted(random_tp50_fp50_dummy_annotations, key=lambda x: (x["bbox"][0], x["bbox"][1]))
print(sorted_random_tp50_fp50_dummy_annotations[0])

{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0}


In [14]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import pycocotools.mask as mask_util

In [15]:
# def evaluate_coco(gt_annotations, pred_annotations, iou_type="bbox", area_rng="all", max_dets=100):
#     """
#     Evaluate COCO-style annotations.
    
#     Args:
#         gt_annotations (list): Ground truth annotations.
#         pred_annotations (list): Predicted annotations.
#         iou_type (str): Type of IoU to evaluate ('bbox', 'segm', 'keypoints').
#         area_rng (str): Area range for evaluation ('all', 'small', 'medium', 'large').
#         max_dets (int): Maximum number of detections to consider.
    
#     Returns:
#         COCOeval: Evaluated COCO object.
#     """
#     coco_gt = COCO()
#     coco_gt.dataset['annotations'] = gt_annotations
#     coco_gt.createIndex()

#     coco_dt = coco_gt.loadRes(pred_annotations)

#     coco_eval = COCOeval(coco_gt, coco_dt, iouType=iou_type)
#     coco_eval.params.areaRng = [coco_eval.params.areaRng[0]]  # Set area range to all
#     coco_eval.params.maxDets = [max_dets]  # Set max detections

#     coco_eval.evaluate()
#     coco_eval.accumulate()
#     coco_eval.summarize()

#     return coco_eval
def evaluate_coco(categories, images, pred_annotations, gt_annotations, iou_type="bbox"):
    """
    Evaluate COCO-style annotations.
    
    Args:
        categories (list): List of categories.
        images (list): List of images.
        pred_annotations (list): Predicted annotations.
        gt_annotations (list): Ground truth annotations.
        iou_type (str): Type of IoU to evaluate ('bbox', 'segm', 'keypoints').
        area_rng (str): Area range for evaluation ('all', 'small', 'medium', 'large').
        max_dets (int): Maximum number of detections to consider.
    
    Returns:
        COCOeval: Evaluated COCO object.
    """
    coco_gt = COCO()
    coco_gt.dataset['categories'] = categories
    coco_gt.dataset['images'] = images
    coco_gt.dataset['annotations'] = gt_annotations
    coco_gt.dataset["info"] = {}
    coco_gt.dataset["licenses"] = []
    coco_gt.createIndex()

    coco_dt = coco_gt.loadRes(pred_annotations)

    coco_eval = COCOeval(coco_gt, coco_dt, iouType=iou_type)
    # print(coco_eval.params.areaRng)
    # print( coco_eval.params.maxDets)
    # coco_eval.params.areaRng = [coco_eval.params.areaRng[0] for i in range(4)]  # Set area range to all
    # coco_eval.params.maxDets = [0, 1, 10]

    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval

In [16]:
coco_eval = evaluate_coco(dummy_categories, dummy_images, correct_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.980
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.990
 Average Recall     (AR) @[ IoU=0

In [17]:
print(coco_eval.stats)

[ 0.98019802  0.98019802  0.98019802  0.98019802 -1.         -1.
  0.          0.09        0.99        0.99       -1.         -1.        ]


In [18]:
coco_eval = evaluate_coco(dummy_categories, dummy_images,tp50_fp50_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.485
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.485
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.485
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.485
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.490
 Average Recall     (AR) @[ IoU=0

In [19]:
coco_eval = evaluate_coco(dummy_categories, dummy_images,random_tp50_fp50_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.278
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.010
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.050
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.490
 Average Recall     (AR) @[ IoU=0

In [20]:
coco_eval = evaluate_coco(dummy_categories, dummy_images,fp50_tp50_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= 

In [21]:
coco_eval = evaluate_coco(dummy_categories, dummy_images,even_tp_od_fp_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.050
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0

In [22]:
coco_eval = evaluate_coco(dummy_categories, dummy_images,fp_30_80_dummy_annotations, gt_dummy_annotations, iou_type="bbox")

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.21s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.090
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.490
 Average Recall     (AR) @[ IoU=0

In [23]:
sorted_random_tp50_fp50_dummy_annotations = sorted(random_tp50_fp50_dummy_annotations, key=lambda x: (x["bbox"][1], x["bbox"][0]))

In [24]:
print(random_tp50_fp50_dummy_annotations[0])

{'id': 1, 'image_id': 0, 'category_id': 1, 'bbox': [20, 40, 10, 10], 'bbox_xyxy': [20, 40, 30, 50], 'area': 100, 'iscrowd': 0, 'score': 1.0, 'segmentation': [[20, 40, 20, 50, 30, 50, 30, 40]]}


In [25]:
print(tp50_fp50_dummy_annotations[0])
print(sorted_random_tp50_fp50_dummy_annotations[0])

{'id': 1, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0, 'segmentation': [[0, 0, 0, 10, 10, 10, 10, 0]]}
{'id': 71, 'image_id': 0, 'category_id': 1, 'bbox': [0, 0, 10, 10], 'bbox_xyxy': [0, 0, 10, 10], 'area': 100, 'iscrowd': 0, 'score': 1.0, 'segmentation': [[0, 0, 0, 10, 10, 10, 10, 0]]}


In [26]:
for s_r_a,t_a in zip(sorted_random_tp50_fp50_dummy_annotations, tp50_fp50_dummy_annotations):
    assert s_r_a["bbox"] == t_a["bbox"], f"BBox mismatch: {s_r_a['bbox']} != {t_a['bbox']}"
    assert s_r_a["bbox_xyxy"] == t_a["bbox_xyxy"], f"BBox XYXY mismatch: {s_r_a['bbox_xyxy']} != {t_a['bbox_xyxy']}"

In [27]:
# Compute COCO AP
images = []
gt_anns = []
dt_anns = []
for image_id in 

SyntaxError: invalid syntax (3398930631.py, line 5)

In [ ]:
import json
def load_json(file_path):
    """
    Load a JSON file and return its content as a Python dictionary.

    Parameters:
        file_path (str): The path to the JSON file.

    Returns:
        dict: The content of the JSON file as a dictionary.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data
def save_json(data, file_path):
    """
    Save a Python dictionary to a JSON file.

    Parameters:
        data (dict): The data to save.
        file_path (str): The path where the JSON file will be saved.
    """
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)